In [2]:
import torch

In [3]:
with open('../data/tinyshakespeare/input.txt','r',encoding='utf-8') as f:
    text = f.read()

In [4]:
print('Total rows in text',len(text))

Total rows in text 1115394


In [5]:
text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [6]:
# build vocabulary table
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f'chars are: {''.join(chars)}')
print(f'vocab_size is: {vocab_size}')

chars are: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab_size is: 65


In [7]:
# build tokenizer
itos ={i:ch for i,ch in enumerate(chars)}
stoi ={ch:i for i,ch in enumerate(chars)}

encode = lambda s:[stoi[c] for c in s]
decode = lambda l:''.join([itos[i] for i in l])

In [8]:
print('hi there:',decode(encode('hi there')))

hi there: hi there


In [9]:
data = torch.tensor(encode(text),dtype=torch.long)
print(f'data shape and dtype are: {data.shape} {data.dtype}')
print(data[:10])

data shape and dtype are: torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [10]:
# 将数据分割成训练集和测试集
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [11]:
# 上下文长度
block_size = 8
# 9个训练数据实际包含了8组训练数据
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input sequence is {context}, the target is {target}')

when input sequence is tensor([18]), the target is 47
when input sequence is tensor([18, 47]), the target is 56
when input sequence is tensor([18, 47, 56]), the target is 57
when input sequence is tensor([18, 47, 56, 57]), the target is 58
when input sequence is tensor([18, 47, 56, 57, 58]), the target is 1
when input sequence is tensor([18, 47, 56, 57, 58,  1]), the target is 15
when input sequence is tensor([18, 47, 56, 57, 58,  1, 15]), the target is 47
when input sequence is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is 58


In [13]:
# build batch，通过batch可以加速训练过程
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [14]:
xb,yb = get_batch('train')
print(f'x is:{xb}')
print(f'y is:{yb}')

print('-'*30)
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b][:t+1]
        target = yb[b][t]
        print(f'when input sequence is {context}, the target is {target}')

x is:tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
y is:tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
------------------------------
when input sequence is tensor([24]), the target is 43
when input sequence is tensor([24, 43]), the target is 58
when input sequence is tensor([24, 43, 58]), the target is 5
when input sequence is tensor([24, 43, 58,  5]), the target is 57
when input sequence is tensor([24, 43, 58,  5, 57]), the target is 1
when input sequence is tensor([24, 43, 58,  5, 57,  1]), the target is 46
when input sequence is tensor([24, 43, 58,  5, 57,  1, 46]), the target is 43
when input sequence is tensor([24, 43, 58,  5, 57,  1, 46, 43]), the target is 39
when input sequence is tensor([44]), the target is 53
when input sequence is te

In [15]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
        
    def forward(self,idx,targets):
        logits = self.token_embedding_table(idx)  #B,T,C
        
        B,T,C = logits.shape
        # 因为pytorch的cross_entropy函数要求输入(mini_batch,C)的格式，以bigram为例，C=vocab_size，实际有B*T个批次，所以需要将logits的形状变为(B*T,C)
        logits = logits.view(B*T,C)
        # targets也做相同的形状变换匹配输入格式
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits,targets)
        
        return logits,loss

In [24]:
m = BigramLanguageModel(vocab_size=vocab_size)
logits,loss = m(xb,yb)
print(logits.shape)

torch.Size([32, 65])


In [ ]:
# 可以看到的loss是4.8786，但是我们可以预估初始的loss，应该是-ln(1/65)=4.1744。
loss

tensor(4.8786, grad_fn=<NllLossBackward0>)